In [21]:
import logging
from itertools import combinations
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import random
from icecream import ic
import heapq

Added properties to the Problem class to simplify data access

In [22]:
class Problem:
    _graph: nx.Graph
    _alpha: float
    _beta: float
    _density: float

    def __init__(
        self,
        num_cities: int,
        *,
        alpha: float = 1.0,
        beta: float = 1.0,
        density: float = 0.5,
        seed: int = 42,
    ):
        rng = np.random.default_rng(seed)
        self._alpha = alpha
        self._beta = beta
        self._density = density
        cities = rng.random(size=(num_cities, 2))
        cities[0, 0] = cities[0, 1] = 0.5

        self._graph = nx.Graph()
        self._graph.add_node(0, pos=(cities[0, 0], cities[0, 1]), gold=0)
        for c in range(1, num_cities):
            self._graph.add_node(c, pos=(cities[c, 0], cities[c, 1]), gold=(1 + 999 * rng.random()))

        tmp = cities[:, np.newaxis, :] - cities[np.newaxis, :, :]
        d = np.sqrt(np.sum(np.square(tmp), axis=-1))
        for c1, c2 in combinations(range(num_cities), 2):
            if rng.random() < density or c2 == c1 + 1:
                self._graph.add_edge(c1, c2, dist=d[c1, c2])

        assert nx.is_connected(self._graph)

    @property
    def graph(self) -> nx.Graph:
        return nx.Graph(self._graph)

    @property
    def dist_dict(self):
        return {(u, v): data['dist'] for u, v, data in self._graph.edges(data=True)}

    @property
    def gold_dict(self):
        return {n: data['gold'] for n, data in self._graph.nodes(data=True)}

    def cost(self, path, weight):
        dist = nx.path_weight(self._graph, path, weight='dist')
        return dist + (self._alpha * dist * weight) ** self._beta

    def baseline(self):
        total_cost = 0
        for dest, path in nx.single_source_dijkstra_path(
            self._graph, source=0, weight='dist'
        ).items():
            cost = 0
            for c1, c2 in zip(path, path[1:]):
                cost += self.cost([c1, c2], 0)
                cost += self.cost([c1, c2], self._graph.nodes[dest]['gold'])
            logging.debug(
                f"dummy_solution: go to {dest} ({' > '.join(str(n) for n in path)} ({cost})"
            )
            total_cost += cost
        return total_cost

    def plot(self):
        plt.figure(figsize=(10, 10))
        pos = nx.get_node_attributes(self._graph, 'pos')
        size = [100] + [self._graph.nodes[n]['gold'] for n in range(1, len(self._graph))]
        color = ['red'] + ['lightblue'] * (len(self._graph) - 1)
        return nx.draw(self._graph, pos, with_labels=True, node_color=color, node_size=size)

In [23]:
logging.getLogger().setLevel(logging.WARNING)

ic(Problem(100, density=0.2, alpha=1, beta=1).baseline())
ic(Problem(100, density=0.2, alpha=2, beta=1).baseline())
ic(Problem(100, density=0.2, alpha=1, beta=2).baseline())
ic(Problem(100, density=1, alpha=1, beta=1).baseline())
ic(Problem(100, density=1, alpha=2, beta=1).baseline())
ic(Problem(100, density=1, alpha=1, beta=2).baseline())
ic(Problem(1_000, density=0.2, alpha=1, beta=1).baseline())
ic(Problem(1_000, density=0.2, alpha=2, beta=1).baseline())
ic(Problem(1_000, density=0.2, alpha=1, beta=2).baseline())
ic(Problem(1_000, density=1, alpha=1, beta=1).baseline())
ic(Problem(1_000, density=1, alpha=2, beta=1).baseline())
ic(Problem(1_000, density=1, alpha=1, beta=2).baseline())

None

ic| Problem(100, density=0.2, alpha=1, beta=1).baseline(): np.float64(25266.40561851072)
ic| Problem(100, density=0.2, alpha=2, beta=1).baseline(): np.float64(50425.30961817918)
ic| Problem(100, density=0.2, alpha=1, beta=2).baseline(): np.float64(5334401.927002504)
ic| Problem(100, density=1, alpha=1, beta=1).baseline(): np.float64(18266.18579582672)
ic| Problem(100, density=1, alpha=2, beta=1).baseline(): np.float64(36457.918462372065)
ic| Problem(100, density=1, alpha=1, beta=2).baseline(): np.float64(5404978.08899582)
ic| Problem(1_000, density=0.2, alpha=1, beta=1).baseline(): np.float64(195402.95810394012)
ic| Problem(1_000, density=0.2, alpha=2, beta=1).baseline(): np.float64(390028.72126288974)
ic| Problem(1_000, density=0.2, alpha=1, beta=2).baseline(): np.float64(37545927.70213464)
ic| Problem(1_000, density=1, alpha=1, beta=1).baseline(): np.float64(192936.23377726765)
ic| Problem(1_000, density=1, alpha=2, beta=1).baseline(): np.float64(385105.64149576554)
ic| Problem(1_000

#### COST FUNCTION


In [24]:
def calculate_full_path_cost_final(problem_instance, path, trip_counts=None):
    # Calculate total path cost considering weight accumulation and trip distribution
    # Dividing gold across multiple trips reduces weight penalty when beta > 1
    total_cost = 0.0
    current_weight = 0.0
    infeasible_nodes = []
    
    # Extract distance and gold data from problem
    dist_dict = problem_instance.dist_dict
    gold_dict = problem_instance.gold_dict
    
    # Count number of trips (returns to depot) in path
    total_trips_in_path = 0
    for i in range(len(path) - 1):
        if path[i+1][0] == 0:
            total_trips_in_path += 1
    
    # Use default trip counts if not provided
    if trip_counts is None:
        trip_counts = [1] * total_trips_in_path
    
    trip_index = 0
    current_trip_count = trip_counts[trip_index] if trip_index < len(trip_counts) else 1
    
    # Traverse path and accumulate costs
    for i in range(len(path) - 1):
        u = path[i][0]
        v = path[i + 1][0]
        u_flag = path[i][1]
        
        this_segment_trips = current_trip_count

        # Accumulate gold weight if node is active
        if u != 0 and u_flag == True:
            gold_at_u = gold_dict.get(u, 0)
            current_weight += gold_at_u / this_segment_trips
        
        # Get edge distance
        dist_uv = dist_dict.get((min(u, v), max(u, v)))
        if dist_uv is None:
            dist_uv = 1.0
        
        # Compute segment cost with weight penalty
        cost_segment_unit = dist_uv + (problem_instance._alpha * dist_uv * current_weight) ** problem_instance._beta
        total_cost += cost_segment_unit * this_segment_trips

        # Reset weight at depot and advance trip counter
        if v == 0:
            current_weight = 0.0
            
            trip_index += 1
            if trip_index < len(trip_counts):
                current_trip_count = trip_counts[trip_index]
            else:
                current_trip_count = 1
        
        # Reset weight at start node        
        if u == 0:
            current_weight = 0.0
    
    return (len(infeasible_nodes), total_cost)

#### POPULATION CREATION

Built a greedy population by computing shortest paths from depot to all nodes. Randomly select nodes to visit and collect gold, creating diverse solutions with different point where i collecyt the gold. This generates multiple individual solutions for GA initialization.

In [25]:
def build_nearest_neighbors_cache(P, k=5):
    # Build cache of k-nearest neighbors per node for efficient mutation candidate selection
    # Avoids recomputing distances during mutation iterations
    dist_dict = P.dist_dict
    graph = P.graph
    distance_near_cache = {}
    
 
    for node in graph.nodes():
        neighbors_with_dist = []
        for (u, v), dist in dist_dict.items():
            if u == node:
                neighbors_with_dist.append((v, dist))
            elif v == node:
                neighbors_with_dist.append((u, dist))
        
        neighbors_with_dist.sort(key=lambda x: x[1])
        distance_near_cache[node] = {
            neighbor: dist for neighbor, dist in neighbors_with_dist[:k]
        }
    
    return distance_near_cache



In [26]:
def neighborhood_greedy_strategy_dijistra(problem_instance):
    # Compute shortest Dijkstra paths from depot to all nodes
    # Used for greedy population initialization and segment reconstruction
    graph = problem_instance.graph
    
    node_to_path = dict(nx.single_source_dijkstra_path(graph, source=0, weight='dist'))

    path_list = [node_to_path[node] for node in sorted(node_to_path.keys())]

    return path_list

In [27]:
def choice_a_path(path_list, seed=42):
    # Initialize empty path and RNG
    full_path = []
    rng = random.Random(seed)
    
    # Track nodes not yet visited for gold collection
    ungolden_nodes = [i for i in range(1, len(path_list))]
    
    # Build path by randomly selecting unvisited nodes
    while ungolden_nodes:
        # Randomly choose next node to visit
        node = rng.choice(ungolden_nodes)
        
        # Get shortest path from depot to chosen node
        path_for_node = list(path_list[node].copy())
        
        # Add path from depot (without destination) with inactive flag
        for p in path_for_node[:-1]:
            full_path.append((p, False))
        
        # Reverse path to go back from destination
        path_for_node.reverse()

        # Add reversed path: mark visited nodes with True flag
        for p in (path_for_node[:-1]):
            if p in ungolden_nodes:
                full_path.append((p, True))
                ungolden_nodes.remove(p)
            else:
                full_path.append((p, False))
    
    # Append return to depot
    full_path.append((0, False))
    
    return full_path


def create_population(problem_instance, population_size):
    population = []
    near_list=neighborhood_greedy_strategy_dijistra(problem_instance=problem_instance)
    if problem_instance._density<2:
        for n in range(population_size):
            individual=choice_a_path(near_list, seed=n)
            population.append(individual)
        return population
    else:
        return population



#### MUTATION AND CROSSOVER

Implemented two mutations and one crossover operator that intelligently reduce cost:

**Insertion Mutation**: Insert nearest-neighbor nodes into random path segments to collect additional gold with minimal distance increase.

**Trip Count Mutation**: Distribute gold across multiple trips to reduce weight penalties when beta > 1. Multiply trip counts on random segments to test cost reduction.

**Crossover**: Replace path segments between individuals while maintaining solution validity.

All operators include duplicate detection and return delta costs for efficient fitness calculation without recomputing entire path cost.

In [28]:
def find_node_with_flag_true(path, wanted_node):
    # Find index of node with active flag in path
    for i in range(len(path)):
        if path[i][1] and path[i][0] == wanted_node:
            return i
    return -1

def founding_start_and_end_index(path, start_index):
    # Find trip boundaries: start and end indices where node==0
    end_index=start_index+1
    while start_index > 0 and path[start_index][0]!=0:
        start_index-=1
    while end_index < len(path) - 1 and path[end_index][0]!=0:
        end_index+=1

    return start_index, end_index

def find_True_flags(path_segment):
    # Extract all nodes with active flag (True) from segment
    gold_elements=[]
    for node, flag in path_segment:
        if flag:
            gold_elements.append(node)

    return gold_elements

def remove_more_gold_nodes(parent_path, gold_nodes, start_index_2, end_index_2):
    # Remove gold node from path and update boundary indices
    new_path=parent_path.copy()
    # Locate node with active flag
    index=find_node_with_flag_true(new_path, gold_nodes)
    
    # Find trip segment containing node
    start_index, end_index = founding_start_and_end_index(new_path, index)
    list_of_gold_nodes_in_segment = find_True_flags(new_path[start_index:end_index])
    
    # If only gold node in segment, remove entire trip
    if len(list_of_gold_nodes_in_segment)==1:
        new_path=new_path[:start_index]+new_path[end_index:]
        # Adjust boundaries after deletion
        delta=end_index-start_index
        start_index_2=start_index_2-delta if start_index_2 > start_index else start_index_2
        end_index_2=end_index_2-delta if end_index_2 > end_index else end_index_2
        return new_path, start_index_2, end_index_2
    
    # Otherwise deactivate flag only
    new_path[index]=(new_path[index][0], False)

    return new_path, start_index_2, end_index_2


def find_path_removed(parent_path, gold_nodes):
    # Find and return path segment before removing gold node
    new_path=parent_path.copy()
    # Locate node with active flag
    index=find_node_with_flag_true(new_path, gold_nodes)
    
    # Find trip segment containing node
    start_index, end_index = founding_start_and_end_index(new_path, index)
    list_of_gold_nodes_in_segment = find_True_flags(new_path[start_index:end_index])
    old_insert=new_path[start_index:end_index+1]
    
    # If only gold node in segment, remove entire trip
    if len(list_of_gold_nodes_in_segment)==1:
        new_path=new_path[:start_index]+new_path[end_index:]
        new_insert=[]
        return new_path, new_insert, old_insert
    
    # Otherwise deactivate flag and return modified segment
    new_path[index]=(new_path[index][0], False)
    new_insert=new_path[start_index:end_index+1]

    return new_path, new_insert, old_insert


def insert_more_gold_nodes(gold_nodes, path_list):
    # Build path for specific gold nodes using greedy strategy
    full_path = []
    seed = random.randint(0, 1000)
    rng = random.Random(seed)
    ungolden_nodes = gold_nodes.copy()
    
    # Build path by randomly selecting unvisited gold nodes
    while ungolden_nodes:
        # Randomly choose next gold node to visit
        node = rng.choice(ungolden_nodes)
        
        # Get shortest path from depot to chosen node
        path_for_node = list(path_list[node].copy())
        
        # Add path from depot (without destination) with inactive flag
        for p in path_for_node[:-1]:
            full_path.append((p, False))
        
        # Reverse path to go back from destination
        path_for_node.reverse()
        
        # Add reversed path: mark visited gold nodes with True flag
        for p in (path_for_node[:-1]):
            if p in ungolden_nodes:
                full_path.append((p, True))
                ungolden_nodes.remove(p)
            else:
                full_path.append((p, False))
    
    # Append return to depot
    full_path.append((0, False))

    return full_path


def find_segment_by_landmark_nodes(path, landmark_nodes):
    """
    Find segment boundaries in path using landmark nodes (active nodes with flag=True)
    This is more robust than index-based approach since indices change during mutations
    
    Args:
        path: list of (node, flag) tuples
        landmark_nodes: list of nodes with flag=True that mark the segment
    
    Returns:
        (start_index, end_index) tuple, or (None, None) if landmark not found
    """
    if not landmark_nodes:
        return None, None
    
    first_landmark = landmark_nodes[0]
    last_landmark = landmark_nodes[-1]
    
    # Find first occurrence of first landmark with flag=True
    start_idx = None
    for i, (node, flag) in enumerate(path):
        if node == first_landmark and flag:
            start_idx = i
            break
    
    if start_idx is None:
        return None, None
    
    # Find segment boundaries from this active node
    seg_start, seg_end = founding_start_and_end_index(path, start_idx)
    
    return seg_start, seg_end


In [29]:
def apply_insertion(segment, idx_rel, new_node, graph):
    # Insert new_node into segment after node at position idx_rel
    # Computes shortest path from current_node to new_node via Dijkstra
    current_node = segment[idx_rel][0]
    
    path_link = nx.shortest_path(graph, source=current_node, target=new_node, weight='dist')
    
    intermediate_nodes = [(n, False) for n in path_link[1:-1]]
    
    # Insert: prefix + intermediate nodes + new_node with flag=True + suffix
    new_segment = segment[:idx_rel + 1] + intermediate_nodes + [(new_node, True)] + segment[idx_rel + 1:]
    
    return new_segment


def mutation_neighbor_of_next_insertion_only(path, problem_instance, path_list, neighbor_distance_cache, graph):
    # CORE MUTATION OPERATOR: Select random node with flag=True, insert k-nearest neighbor
    # 1. Find random segment with active node (flag=True)
    # 2. Get k-nearest neighbors from cache for current node
    # 3. Filter candidates: exclude depot (0), current_node, and nodes already in segment
    # 4. Insert via apply_insertion() using shortest paths
    # 5. Detect and remove duplicates: deactivate flag outside modified segment bounds
    # 6. Reconstruct segment cleanly using Dijkstra paths if not destroyed
    # Returns (new_path, (infeas_delta, cost_delta)) for efficient GA/HC fitness calculation
    new_path = path.copy()
    
    # Select random position in path and find its segment boundaries (start/end at depot)
    random_index = random.randint(1, len(new_path) - 2)
    start_index, end_index = founding_start_and_end_index(new_path, random_index)
    starting_segment = path[start_index:end_index + 1]
    
    # Find first node with flag=True in segment to determine insertion point
    idx_to_mutate = None
    for i in range(start_index, end_index):
        if path[i][1]:  # Active node (collecting gold)
            idx_to_mutate = i
            break
    
    if idx_to_mutate is None:  # No active nodes to mutate
        return path, (0, 0)
    
    # Get current and next nodes for insertion context
    next_node = new_path[idx_to_mutate + 1][0]
    current_node = new_path[idx_to_mutate][0]
    
    # Retrieve k-nearest neighbors from cache for current_node (insertion source)
    cached_neighbors = neighbor_distance_cache.get(current_node, {})
    if not cached_neighbors:
        return path, (0, 0)
    
    # Extract nodes with flag=True already in this segment
    active_nodes_in_segment = {node for node, flag in path[start_index:end_index + 1] if flag}
    
    # Filter candidates: exclude depot, current node, nodes already collected in this segment
    # Allow nodes from other segments - duplicate detection handles them later
    candidates = []
    for neighbor in cached_neighbors.keys():
        if neighbor != 0 and neighbor != current_node and neighbor not in active_nodes_in_segment:
            candidates.append(neighbor)
    
    if not candidates:  # No valid candidates to insert
        return path, (0, 0)
    
    # Randomly select one of the valid neighbor candidates
    new_node = random.choice(candidates)
    
    # Insert new node into segment using shortest path connection
    new_segment = apply_insertion(
        starting_segment, 
        idx_to_mutate - start_index,  # Convert to relative index within segment
        new_node, 
        graph
    )
    
    # Reconstruct full path with modified segment
    new_path = new_path[:start_index] + new_segment + new_path[end_index + 1:]
    
    # DUPLICATE DETECTION: Check if new_node already exists elsewhere with flag=True
    # Boundary-based approach: if new_node appears with flag=True outside modified segment bounds, it's a duplicate
    new_segment_end = start_index + len(new_segment) - 1
    segments_to_remove = []
    
    i = 0
    while i < len(new_path):
        if new_path[i][0] == new_node and new_path[i][1]:
            # Check if this occurrence is outside the modified segment (duplicate)
            if i < start_index or i > new_segment_end:
                new_path[i] = (new_node, False)  # Deactivate duplicate flag
                
                # Track the segment where duplicate was removed
                if i < len(new_path) - 1:
                    dup_seg_start, dup_seg_end = founding_start_and_end_index(new_path, i)
                    has_active_nodes = any(new_path[j][1] for j in range(dup_seg_start, min(dup_seg_end + 1, len(new_path))))
                    
                    # Mark empty segment for removal
                    if not has_active_nodes:
                        segments_to_remove.append((dup_seg_start, dup_seg_end))
                
                break  # Only one duplicate possible (node inserted exactly once)
        
        i += 1
    
    # SEGMENT CLEANUP - Remove empty segments or reconstruct if needed
    if segments_to_remove:
        # Remove empty segments that no longer contain active nodes
        seg_start, seg_end = segments_to_remove[0]
        new_path = new_path[:seg_start] + new_path[seg_end:]
    else:
        # Reconstruct segment cleanly using Dijkstra paths for valid connections
        # Find first and last active nodes in modified segment
        first_index_true = None
        last_index_true = None
        first_element_true = None
        last_element_true = None
        
        for i in range(start_index, end_index + 1):
            if new_path[i][1]:  # Node with flag=True
                if first_index_true is None:
                    first_index_true = i
                    first_element_true = new_path[i][0]
                last_index_true = i
                last_element_true = new_path[i][0]
        
        if first_element_true is not None:
            seg = []
            
            # Part 1: Depot to first active node using Dijkstra path
            path_to_first = path_list[first_element_true]
            for node in path_to_first[:-1]:  # Exclude destination
                seg.append((node, False))
            seg.append((first_element_true, True))
            
            # Part 2: Intermediate nodes between first and last active nodes
            for i in range(first_index_true + 1, last_index_true):
                seg.append(new_path[i])
            
            # Part 3: Last active node (if different from first)
            if first_element_true != last_element_true:
                seg.append((last_element_true, True))
            
            # Part 4: Last active node back to depot using reversed Dijkstra path
            path_from_last = path_list[last_element_true]
            path_from_last_reversed = list(reversed(path_from_last))
            for node in path_from_last_reversed[1:-1]:  # Exclude last node and depot
                seg.append((node, False))
            seg.append((0, False))
            
            # Replace segment in full path
            new_path = new_path[:start_index] + seg + new_path[end_index + 1:]
        
    # Calculate cost delta: compare full path costs before and after mutation
    infeas_before, cost_before = calculate_full_path_cost_final(problem_instance, path)
    infeas_after, cost_after = calculate_full_path_cost_final(problem_instance, new_path)
    
    delta_infeas = infeas_after - infeas_before
    delta_cost = cost_after - cost_before
    
    return new_path, (delta_infeas, delta_cost)

In [30]:
def mutate_trip_counts(trip_counts):
    # Multiply selected trip count by 4 to test cost reduction
    
    new_counts = trip_counts[:]  
    # Select random trip segment to modify
    idx_to_change = random.randint(0, len(new_counts) - 1)
    
    # Increase trip count to distribute weight across trips
    increment = 4
    new_counts[idx_to_change] *= increment
    
    return new_counts, idx_to_change


def get_trip_boundaries(path, cache=None):
    # Return trip segment boundaries using cache if available
    if cache is not None:
        return cache.get_trip_boundaries()
    
    # Identify depot returns to find trip boundaries
    boundaries = []
    trip_start = 0
    
    # Mark (start, end) indices for each trip (returns to depot at node 0)
    for i, (node, _) in enumerate(path):
        if node == 0 and i > 0:
            boundaries.append((trip_start, i))
            trip_start = i
    
    # Add final trip if path doesn't end at depot
    if trip_start < len(path):
        boundaries.append((trip_start, len(path) - 1))
    
    return boundaries

def evaluate_trip_mutation_smart(problem_instance, path, old_counts, new_counts, changed_idx):
    # Evaluate cost change for modified trip count on segment only
    boundaries = get_trip_boundaries(path)
    start_idx, end_idx = boundaries[changed_idx]
    
    # Extract only affected trip segment for evaluation
    mini_path = path[start_idx : end_idx + 1]
    
    # Compute costs with old and new trip counts
    old_trip_count_list = [old_counts[changed_idx]]
    new_trip_count_list = [new_counts[changed_idx]]
    
    # Calculate cost delta on sub-path only (efficient evaluation)
    _, cost_old = calculate_full_path_cost_final(problem_instance, mini_path, old_trip_count_list)
    _, cost_new = calculate_full_path_cost_final(problem_instance, mini_path, new_trip_count_list)
    
    delta = cost_new - cost_old
    
    return delta


In [31]:
def crossover_zero_paths_with_delta(parent1, parent2, possible_paths, problem_instance):
    # Crossover operator: replace parent2 segment with parent1 segment centered on shared gold node
    # 1. Extract random segment from parent1 containing active gold node
    # 2. Find corresponding segment in parent2 with same gold node
    # 3. Replace segment in parent2 with parent1 segment
    # 4. Handle conflicting gold nodes: remove from other trips if already collected elsewhere
    # 5. Insert missing gold nodes from parent2 that aren't in parent1 segment
    # 6. Return new offspring with delta cost for fitness calculation
    # Args:
    #   parent1, parent2: path solutions to cross
    #   possible_paths: precomputed Dijkstra paths from depot
    #   problem_instance: problem definition with graph and parameters
    # Returns: (offspring_path, (infeas_delta, cost_delta, infeas_nodes))
    
    p2_working = parent2.copy()
    
    # Select random segment from parent1 bounded by depot (node 0)
    start_index_1 = random.randint(1, len(parent1) - 2)
    start_index_1, end_index_1 = founding_start_and_end_index(parent1, start_index_1)
    
    segment_p1 = parent1[start_index_1 : end_index_1 + 1]
    
    # Find first active node (flag=True) in parent1 segment - this is the crossing point
    gold_element = 0
    index_gold_element_1 = -1
    for i in range(start_index_1, end_index_1):
        if parent1[i][1] == True:
            gold_element = parent1[i][0]
            index_gold_element_1 = i
            break
            
    # If no active node found, return parent2 unchanged
    if gold_element == 0:
        return p2_working, (0, 0, [])

    # Collect all other active nodes in parent1 segment (after gold_element)
    list_gold_1 = find_True_flags(parent1[index_gold_element_1+1 : end_index_1])

    # Find same gold node in parent2 and locate its segment boundaries
    index_gold_element_2 = find_node_with_flag_true(p2_working, gold_element)
    
    try:
        start_index_2, end_index_2 = founding_start_and_end_index(p2_working, index_gold_element_2)
    except (IndexError, TypeError):
        return p2_working, (0, 0, [])

    # Store original parent2 segment for cost comparison
    segment_p2_original = p2_working[start_index_2 : end_index_2 + 1]
    
    # Calculate cost of parent1 segment vs parent2 segment (initial delta)
    res_seg1 = calculate_full_path_cost_final(problem_instance, segment_p1)
    res_seg2 = calculate_full_path_cost_final(problem_instance, segment_p2_original)
    
    delta_infeas = res_seg1[0] - res_seg2[0]
    delta_cost = res_seg1[1] - res_seg2[1]
    infeas_nodes = []
    if len(res_seg1) > 2: infeas_nodes.extend(res_seg1[2])

    # Identify active nodes in each parent segment
    list_gold_2 = find_True_flags(p2_working[start_index_2 : end_index_2])
    
    # Handle conflicting nodes: if parent1 has active nodes that parent2 doesn't have in crossing segment
    if len(list_gold_1) > 0:
        for node in list_gold_1:
            # If node from parent1 exists elsewhere in parent2 (duplicate), remove it
            if node not in list_gold_2:
                
                idx_external = find_node_with_flag_true(p2_working, node)
                
                if idx_external != -1:
                    # Calculate cost before removing duplicate
                    s_ext, e_ext = founding_start_and_end_index(p2_working, idx_external)
                    seg_ext_old = p2_working[s_ext : e_ext + 1]
                    res_ext_old = calculate_full_path_cost_final(problem_instance, seg_ext_old)
                    
                    # Remove the duplicate node from other trip
                    p2_working, start_index_2, end_index_2 = remove_more_gold_nodes(
                        p2_working, node, start_index_2, end_index_2
                    )
                    
                    # Calculate cost after removal for delta update
                    check_idx = min(idx_external, len(p2_working)-2)
                    s_ext_new, e_ext_new = founding_start_and_end_index(p2_working, check_idx)
                    seg_ext_new = p2_working[s_ext_new : e_ext_new + 1]
                    res_ext_new = calculate_full_path_cost_final(problem_instance, seg_ext_new)
                    
                    # Accumulate cost delta from removal
                    delta_infeas += (res_ext_new[0] - res_ext_old[0])
                    delta_cost += (res_ext_new[1] - res_ext_old[1])
                    if len(res_ext_new) > 2: infeas_nodes.extend(res_ext_new[2])

    # Remove crossing node from parent2 list to avoid counting twice
    if gold_element in list_gold_2:
        list_gold_2.remove(gold_element)
    
    # Find nodes in parent2 that are not in parent1 segment (need to be inserted)
    list_2_not_in_1 = [node for node in list_gold_2 if node not in list_gold_1]

    # Insert missing nodes from parent2 into new path
    new_path_append = []
    if len(list_2_not_in_1) > 0:
        new_path_append = insert_more_gold_nodes(list_2_not_in_1, possible_paths)
        
        # Calculate cost of inserted segment
        res_append = calculate_full_path_cost_final(problem_instance, new_path_append)
        
        # Add to delta cost
        delta_infeas += res_append[0]
        delta_cost += res_append[1]
        if len(res_append) > 2: infeas_nodes.extend(res_append[2])

    # Construct new individual: parent2 prefix + parent1 segment + parent2 suffix + inserted nodes
    new_individual = (
        p2_working[:start_index_2] + 
        segment_p1 +
        p2_working[end_index_2+1:] +
        new_path_append[1:]
    )
    
    return new_individual, (delta_infeas, delta_cost, infeas_nodes)

#### GENETIC ALGORITHM AND HILL CLIMBING

**Hybrid Approach**: Density-based algorithm selection

**Sparse Networks (density < 0.8)**: Use Genetic Algorithm with doubled population size for solution space exploration. Tournament size and crossover probability adapt during generations for improved convergence.

**Dense Networks (density >= 0.8)**: Use Hill Climbing from greedy initialization, which converges faster when solution diversity is limited.

**Trip Count Optimization**: When beta > 1 (weight penalty is significant), apply hill climbing to optimize trip count distribution per segment, reducing total cost by dividing gold across multiple trips.

In [32]:

def tournament_selection(population_with_fitness, tournament_size=3):
    candidates = random.sample(population_with_fitness, tournament_size)
    
    winner = min(candidates, key=lambda x: (x[1][0], x[1][1]))
    
    return winner[0], winner[1]

In [33]:
def hill_climbing(problem_instance, initial_solution, n_iterations=1000):
    # Hill climbing optimizer: iteratively improve solution by accepting only better moves
    # Used for dense graphs (density >= 0.8) where GA population diversity is limited
    current_solution = initial_solution.copy()
    
    curr_infeas, curr_cost = calculate_full_path_cost_final(problem_instance, current_solution)
    
    neighbor_distance_cache = build_nearest_neighbors_cache(problem_instance, k=5)
    path_list = neighborhood_greedy_strategy_dijistra(problem_instance)
    graph = problem_instance.graph
    
    for iteration in range(n_iterations):
        
        neighbor_solution, delta_tuple = mutation_neighbor_of_next_insertion_only(
            current_solution, problem_instance, path_list, neighbor_distance_cache, graph
        )
        
        delta_infeas = delta_tuple[0]
        delta_cost = delta_tuple[1]
        
        accept = False
        
        if delta_infeas < 0:
            accept = True
            
        elif delta_infeas > 0:
            accept = False
            
        else:
            if delta_cost <= 0:
                accept = True
            else:
                accept = False
        
        
        if accept:
            current_solution = neighbor_solution
            curr_infeas += delta_infeas
            curr_cost += delta_cost

    return current_solution, curr_cost

In [34]:
def genetic_algorithm(problem_instance, population_size=100, n_generations=50, temperature=0.8):
    # Genetic Algorithm with adaptive parameters:
    # - Tournament size increases over generations (2->6) for gradual convergence pressure
    # - Crossover probability adapts: base 0.1 + (temperature * progress * 0.8)
    # - Elitism preserves best individual each generation
    # - Both mutation and crossover use delta-based cost updates for efficiency
    neighbor_distance_cache = build_nearest_neighbors_cache(problem_instance, k=5)
    graph = problem_instance.graph 
    
    raw_population = create_population(problem_instance, population_size)
    population_data = []
    
    for ind in raw_population:
        fit = calculate_full_path_cost_final(problem_instance, ind)
        population_data.append((ind, fit))
        
    path_list = neighborhood_greedy_strategy_dijistra(problem_instance)

    def is_better(fit_a, fit_b):
        if fit_a[0] != fit_b[0]:
            return fit_a[0] < fit_b[0]
        return fit_a[1] < fit_b[1]

    for generation in range(n_generations):
        new_population_data = []
        
        progress = generation / n_generations
        
        min_tournament_k = 2
        max_tournament_k = 6
        current_tournament_k = int(min_tournament_k + progress * (max_tournament_k - min_tournament_k))
        current_tournament_k = max(2, min(6, current_tournament_k))
        
        base_crossover_rate = 0.1
        max_added_rate = 0.8
        current_crossover_prob = base_crossover_rate + (progress * temperature * max_added_rate)
        current_crossover_prob = max(0.0, min(1.0, current_crossover_prob))

        best_of_gen = min(population_data, key=lambda x: (x[1][0], x[1][1]))
        new_population_data.append(best_of_gen)
        
        while len(new_population_data) < population_size:
            parent1_path, p1_fit = tournament_selection(population_data, current_tournament_k)
            parent2_path, p2_fit = tournament_selection(population_data, current_tournament_k)
            
            winner_path = None
            winner_fit = None
            
           
            if random.random() > current_crossover_prob:
                child_path, mut_delta = mutation_neighbor_of_next_insertion_only(
                    parent1_path, problem_instance, path_list, neighbor_distance_cache, graph
                )
                
                m_infeas = p1_fit[0] + mut_delta[0]
                m_cost = p1_fit[1] + mut_delta[1]
                child_fit = (m_infeas, m_cost, [])

                if is_better(child_fit, p1_fit):
                    winner_path, winner_fit = child_path, child_fit
                else:
                    winner_path, winner_fit = parent1_path, p1_fit

            else:
                child_path, delta_tuple = crossover_zero_paths_with_delta(
                    parent1_path, parent2_path, path_list, problem_instance
                )
                
                child_infeas = p2_fit[0] + delta_tuple[0]
                child_cost = p2_fit[1] + delta_tuple[1]
                child_fit = (child_infeas, child_cost, []) 
                
                if is_better(child_fit, p2_fit):
                    winner_path, winner_fit = child_path, child_fit
                else:
                    winner_path, winner_fit = parent2_path, p2_fit
            
            
            new_population_data.append((winner_path, winner_fit))
        
        population_data = new_population_data
        
        if generation % 10 == 0 or generation == n_generations - 1:
            best_now = min(population_data, key=lambda x: (x[1][0], x[1][1]))
            print(f"Gen {generation} | K={current_tournament_k} | CrossoverRate: {current_crossover_prob:.2f} | Best: {best_now[1][1]:.2f}")

    return min(population_data, key=lambda x: (x[1][0], x[1][1]))

In [35]:
def run_hill_climbing_trips(problem_instance, path, initial_trip_counts, max_iter=1000):
    # Initialize trip counts and compute initial cost
    current_counts = list(initial_trip_counts)
    _, current_total_cost = calculate_full_path_cost_final(problem_instance, path, current_counts)
    
    iteration = 0
    improvements = 0
    scaler = 8
    
    # Try different scalers to initialize trip counts efficiently
    for i in range(2):
        # Multiply each element by scaler and convert to int
        new_counts = [int(c * scaler) for c in current_counts]
        _, new_cost = calculate_full_path_cost_final(problem_instance, path, new_counts)
        
        if new_cost < current_total_cost:
            current_counts = new_counts
            current_total_cost = new_cost
        else:
            scaler = scaler / 2
        
    # Hill climbing loop: optimize trip counts
    while iteration < max_iter:
        iteration += 1
        
        # Generate neighbor solution by multiplying random trip count
        new_counts, changed_idx = mutate_trip_counts(current_counts)
        
        # Evaluate cost change on affected segment only
        delta = evaluate_trip_mutation_smart(
            problem_instance, 
            path, 
            current_counts,
            new_counts, 
            changed_idx
        )
        
        # Accept improvement if cost delta is negative
        if delta < 0:
            current_counts = new_counts
            current_total_cost += delta
            improvements += 1

    # Print optimization results
    print(f"\n--- Fine Ottimizzazione ---")
    print(f"Costo Finale: {current_total_cost:.4f}")
    print(f"Miglioramenti trovati: {improvements}")
    print(f"Counts Finali: {current_counts}")
    
    return current_counts, current_total_cost


At last, I create a main function that set the right parameter to solve all the problem in a small amount of time (maximum 5 minutes) and convert it in the format requeried by the project

In [36]:
def conversion_solution(problem_instance,full_path, trip_counts):
    converted_path = []
    list_of_bounds=get_trip_boundaries(full_path)
    for trip_idx in range(len(trip_counts)):
        start_idx, end_idx = list_of_bounds[trip_idx]
        trip_count = trip_counts[trip_idx]
        for num_trips in range(trip_count):
            for i in range(start_idx, end_idx):
                node, is_active = full_path[i]
                if is_active:
                    gold_dict = problem_instance.gold_dict 
                    total_gold = gold_dict[node]
                    gold_amount = total_gold / trip_count if trip_count > 0 else 0
                    converted_path.append((node, gold_amount))
                else:
                    converted_path.append((node, 0))

    converted_path.append((0, 0))
    converted_path=converted_path[1:]
    return converted_path

In [ ]:
def adaptive_solver(problem_instance, n_trip_count_hc=500, fast=True):
    # MAIN ORCHESTRATOR: Density-based algorithm selection
    # Sparse (density < 0.8): Genetic Algorithm with doubled population for diversity
    # Dense (density >= 0.8): Hill Climbing from greedy initialization (faster convergence)
    # Automatically adjusts parameters based on problem size and beta factor
    
    # Clamp problem size to reasonable bounds
    n = len(problem_instance.graph.nodes())
    n = max(100, min(n, 1000))

    # Compute adaptive parameters based on problem size
    population_size = int((30 - (n - 100) / 60) / 2)
    n_generations = int(80 - (n - 100) / 18)
    n_trip_count_hc = int(n)

    if n >= 800:
        n_trip_count_hc = n_trip_count_hc / 2

    # Double population for sparse graphs (more diversity needed for exploration)
    if problem_instance._density < 0.8:
        population_size *= 2

    # Reduce generations for large, dense, high-beta problems (faster convergence)
    if n > 500 and fast and problem_instance._beta > 1:
        n_generations = int(n_generations / 2)

    # DENSITY-BASED ALGORITHM SELECTION
    if problem_instance._density < 0.8:
        # Sparse: GA explores diverse solution space effectively
        ga_result = genetic_algorithm(problem_instance, population_size, n_generations)
        best_path = ga_result[0]
    else:
        # Dense: HC with greedy initialization converges faster than diverse GA
        path_list = neighborhood_greedy_strategy_dijistra(problem_instance)
        initial_path = choice_a_path(path_list, seed=42)
        best_path, _ = hill_climbing(problem_instance, initial_path, n_iterations=population_size * n_generations)

    # TRIP OPTIMIZATION: Divide gold across multiple trips to reduce weight penalty
    # Only optimizes when beta > 1 (weight penalty is significant)
    initial_trip_counts = [1 for _ in range(len(get_trip_boundaries(best_path)))]
    
    if problem_instance._beta > 1:
        # Hill climbing on trip counts: multiply counts to reduce total weight penalty
        optimized_trip_counts, final_cost = run_hill_climbing_trips(
            problem_instance, 
            best_path, 
            initial_trip_counts, 
            max_iter=n_trip_count_hc
        )
    else:
        # Beta <= 1: no weight penalty benefit from multiple trips
        optimized_trip_counts = initial_trip_counts
        _, final_cost = calculate_full_path_cost_final(problem_instance, best_path, optimized_trip_counts)
    
    return best_path, optimized_trip_counts, final_cost

In [38]:
P=Problem(800, density=0.7, alpha=1, beta=2)
x=adaptive_solver(P)
print(x)
print(P.baseline())

#funzione che conta quanti True ho
def count_true_flags(path):
    return sum(1 for _, flag in path if flag)

count_true_flags(x[0])

Gen 0 | K=2 | CrossoverRate: 0.10 | Best: 42771518.67


KeyboardInterrupt: 

In [ ]:
def validate_solution_uniqueness(path):
    # VALIDATION: Ensure each node with flag=True appears exactly once (no duplicates)
    # Critical invariant: gold collected from each node only once
    nodes_with_true = []
    
    for node, flag in path:
        if flag:
            nodes_with_true.append(node)
    
    from collections import Counter
    node_counts = Counter(nodes_with_true)
    violations = [node for node, count in node_counts.items() if count > 1]
    is_valid = len(violations) == 0
    
    return is_valid, violations


def print_solution_validation(path, context=""):
    # LOGGING: Print warning if solution has duplicate nodes with flag=True
    # Silent on success (clean verification)
    is_valid, violations = validate_solution_uniqueness(path)
    
    if not is_valid:
        print("\n" + "="*100)
        print(f"⚠️  WARNING: DUPLICATE NODES WITH FLAG=TRUE DETECTED IN {context}")
        print("="*100)
        print(f"\nNodes appearing multiple times with flag=True: {violations}")
        
        # Show duplicate positions
        for node in violations:
            positions = [i for i, (n, flag) in enumerate(path) if n == node and flag]
            print(f"  Node {node}: positions {positions}")
        
        print("\n" + "="*100 + "\n")
    
    return is_valid


In [42]:

# === VALUTAZIONE COMPLETA DELL'ADAPTIVE SOLVER ===
import pandas as pd
import time

print("="*120)
print("VALUTAZIONE ADAPTIVE SOLVER - COMPREHENSIVE TEST")
print("="*120)

# Parametri di test
densities = [0.3, 0.8, 1.0]  # 4 densità
sizes = [100, 600, 1000]      # 4 grandezze
betas = [0.5, 2.0]             # 3 beta

results = []

total_problems = len(densities) * len(sizes) * len(betas)
current_problem = 0

print(f"\nRisoluzione di {total_problems} problemi...\n")

for density in densities:
    for size in sizes:
        for beta in betas:
            current_problem += 1
            
            # Crea il problema
            P = Problem(size, density=density, alpha=1.0, beta=beta, seed=42)
            
            # Calcola la baseline
            baseline_cost = P.baseline()
            
            # Risolvi con adaptive_solver
            start_time = time.time()
            best_path, trip_counts, solver_cost = adaptive_solver(P, n_trip_count_hc=500, fast=True)
            elapsed_time = time.time() - start_time
            
            # Calcola il miglioramento
            improvement = ((baseline_cost - solver_cost) / baseline_cost) * 100
            
            # Salva i risultati
            results.append({
                'Density': density,
                'Size': size,
                'Beta': beta,
                'Baseline': baseline_cost,
                'Solver': solver_cost,
                'Improvement (%)': improvement,
                'Time (s)': elapsed_time
            })
            
            # Stampa progresso
            print(f"[{current_problem:2d}/{total_problems}] Size={size:4d}, Density={density:.1f}, Beta={beta:.1f} | "
                  f"Baseline={baseline_cost:10.2f}, Solver={solver_cost:10.2f}, "
                  f"Improvement={improvement:6.2f}%, Time={elapsed_time:6.2f}s")

print("\n" + "="*120)
print("RISULTATI COMPLETI")
print("="*120)

# Crea un DataFrame
df = pd.DataFrame(results)

# Formatta i risultati
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("\n" + df.to_string(index=False))

# Statistiche di riepilogo
print("\n" + "="*120)
print("STATISTICHE DI RIEPILOGO")
print("="*120)
print(f"\nTempo medio per problema: {df['Time (s)'].mean():.2f}s")
print(f"Tempo totale: {df['Time (s)'].sum():.2f}s")
print(f"\nMiglioramento medio: {df['Improvement (%)'].mean():.2f}%")
print(f"Miglioramento minimo: {df['Improvement (%)'].min():.2f}%")
print(f"Miglioramento massimo: {df['Improvement (%)'].max():.2f}%")

print("\n" + "="*120)
print("PERFORMANCE PER DENSITÀ")
print("="*120)
by_density = df.groupby('Density')[['Improvement (%)', 'Time (s)']].agg(['mean', 'min', 'max'])
print(by_density)

print("\n" + "="*120)
print("PERFORMANCE PER GRANDEZZA")
print("="*120)
by_size = df.groupby('Size')[['Improvement (%)', 'Time (s)']].agg(['mean', 'min', 'max'])
print(by_size)

print("\n" + "="*120)
print("PERFORMANCE PER BETA")
print("="*120)
by_beta = df.groupby('Beta')[['Improvement (%)', 'Time (s)']].agg(['mean', 'min', 'max'])
print(by_beta)

print("\n" + "="*120)


VALUTAZIONE ADAPTIVE SOLVER - COMPREHENSIVE TEST

Risoluzione di 18 problemi...

Gen 0 | K=2 | CrossoverRate: 0.10 | Best: 1640.67
Gen 10 | K=2 | CrossoverRate: 0.18 | Best: 1615.08
Gen 20 | K=3 | CrossoverRate: 0.26 | Best: 1593.76
Gen 30 | K=3 | CrossoverRate: 0.34 | Best: 1579.29
Gen 40 | K=4 | CrossoverRate: 0.42 | Best: 1563.95
Gen 50 | K=4 | CrossoverRate: 0.50 | Best: 1546.94
Gen 60 | K=5 | CrossoverRate: 0.58 | Best: 1538.42
Gen 70 | K=5 | CrossoverRate: 0.66 | Best: 1531.87
Gen 79 | K=5 | CrossoverRate: 0.73 | Best: 1531.87
[ 1/18] Size= 100, Density=0.3, Beta=0.5 | Baseline=   1884.77, Solver=   1531.87, Improvement= 18.72%, Time=  4.93s
Gen 0 | K=2 | CrossoverRate: 0.10 | Best: 5165937.10
Gen 10 | K=2 | CrossoverRate: 0.18 | Best: 4842185.28
Gen 20 | K=3 | CrossoverRate: 0.26 | Best: 4616931.86
Gen 30 | K=3 | CrossoverRate: 0.34 | Best: 4517541.02
Gen 40 | K=4 | CrossoverRate: 0.42 | Best: 4369227.21
Gen 50 | K=4 | CrossoverRate: 0.50 | Best: 4250767.99
Gen 60 | K=5 | Crosso

KeyboardInterrupt: 

In [ ]:

# === DELTA CALCULATION VERIFICATION TEST ===
import numpy as np

print("="*120)
print("DELTA CALCULATION VERIFICATION - Testing dual-segment delta computation")
print("="*120)

# Create a small test problem
P_test = Problem(50, density=0.7, alpha=1.0, beta=1.5, seed=42)
path_list_test = neighborhood_greedy_strategy_dijistra(P_test)
neighbor_cache_test = build_nearest_neighbors_cache(P_test, k=5)
graph_test = P_test.graph

# Create initial solution
initial_path_test = choice_a_path(path_list_test, seed=100)
initial_cost_before = calculate_full_path_cost_final(P_test, initial_path_test)[1]

print(f"\nInitial path cost: {initial_cost_before:.4f}")
print(f"Initial path length: {len(initial_path_test)}")
print(f"Number of True flags: {sum(1 for _, flag in initial_path_test if flag)}")

# Apply mutation multiple times and verify delta calculation
num_mutations = 10
correct_deltas = 0
incorrect_deltas = 0

print("\n" + "-"*120)
print("Running mutations and verifying delta calculations...")
print("-"*120)

for mutation_num in range(num_mutations):
    current_path = initial_path_test.copy()
    
    # Calculate cost before mutation
    _, cost_before_full = calculate_full_path_cost_final(P_test, current_path)
    
    # Apply mutation
    mutated_path, (delta_infeas, delta_cost) = mutation_neighbor_of_next_insertion_only(
        current_path, P_test, path_list_test, neighbor_cache_test, graph_test
    )
    
    # Calculate cost after mutation (full path)
    _, cost_after_full = calculate_full_path_cost_final(P_test, mutated_path)
    
    # Calculate actual delta (ground truth)
    actual_delta_cost = cost_after_full - cost_before_full
    
    # Check if mutation had any effect
    if delta_cost == 0:
        print(f"\n[{mutation_num+1:2d}] No mutation applied (no valid candidates)")
        continue
    
    # Calculate error in delta
    delta_error = abs(delta_cost - actual_delta_cost)
    relative_error = delta_error / abs(actual_delta_cost) if actual_delta_cost != 0 else 0
    
    # Verify delta is correct
    is_correct = delta_error < 0.01  # Allow small numerical errors
    
    if is_correct:
        correct_deltas += 1
        status = "✓ CORRECT"
    else:
        incorrect_deltas += 1
        status = "✗ INCORRECT"
    
    print(f"\n[{mutation_num+1:2d}] {status}")
    print(f"    Reported delta:     {delta_cost:12.6f}")
    print(f"    Actual delta:       {actual_delta_cost:12.6f}")
    print(f"    Error:              {delta_error:12.6e}")
    print(f"    Relative error:     {relative_error*100:8.4f}%")
    print(f"    Cost before:        {cost_before_full:12.4f}")
    print(f"    Cost after:         {cost_after_full:12.4f}")
    print(f"    Path length change: {len(mutated_path) - len(current_path):+3d}")
    print(f"    True flags change:  {sum(1 for _, flag in mutated_path if flag) - sum(1 for _, flag in current_path if flag):+3d}")

print("\n" + "="*120)
print("SUMMARY")
print("="*120)
print(f"Correct delta calculations:   {correct_deltas}/{num_mutations}")
print(f"Incorrect delta calculations: {incorrect_deltas}/{num_mutations}")
if correct_deltas + incorrect_deltas > 0:
    accuracy = 100 * correct_deltas / (correct_deltas + incorrect_deltas)
    print(f"Accuracy: {accuracy:.1f}%")
print("\n" + "="*120)


DELTA CALCULATION VERIFICATION - Testing dual-segment delta computation

Initial path cost: 137850.4246
Initial path length: 125
Number of True flags: 49

------------------------------------------------------------------------------------------------------------------------
Running mutations and verifying delta calculations...
------------------------------------------------------------------------------------------------------------------------

[ 1] ✗ INCORRECT
    Reported delta:     -10349.745542
    Actual delta:        5240.032852
    Error:              1.558978e+04
    Relative error:     297.5130%
    Cost before:         137850.4246
    Cost after:          143090.4574
    Path length change:  -1
    True flags change:   +0

[ 2] ✗ INCORRECT
    Reported delta:       889.541639
    Actual delta:         890.541639
    Error:              1.000000e+00
    Relative error:       0.1123%
    Cost before:         137850.4246
    Cost after:          138740.9662
    Path length ch